# Story 6 : What Is The State of Food Security and Nutrition in the US

The United Nations Food and Agriculture Organization publication, The State of Food Security and Nutrition in the World 2022 (https://www.fao.org/documents/card/en/c/cc0639en) might lead one to the conclusion that it's an elsewhere problem. That the people who are suffering malnutrition and starvation are "elsewhere", not in our backyard. For this assignment you will need to take a closer look here at home (the US). Are US children suffering these plights?

Notes:

You may use ANY graphical package that you find useful including PowerBI, Tableau, etc.
You will need to locate and source data that reflects food security and nutrition by state broken down by men, women, children and by age groups.
Your analysis should demonstrate correlations that exist between level of poverty and food insecurity, malnutrition and starvation.
Your data and analysis should also indicate what happens to the children as they mature into adults. Will they become fully functional citizens or will they require continued support?
Your data visualizations need to tell the story for a U.S. Senator that you were lobbying to address the issue of food insecurity in the US.

- https://www.census.gov/data/datasets/time-series/demo/cps/cps-supp_cps-repwgt/cps-food-security.html#cpssupps 
- https://www.ers.usda.gov/data-products/food-security-in-the-united-states/

In [32]:
# load packages
import numpy as np # need to downgrade this packages pip install numpy==1.26.4  pyyaml
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import geopandas as gpd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

## Data Preparation

In [33]:
cps_data_url = 'https://raw.githubusercontent.com/eddiexunyc/food_security_us/refs/heads/main/Resources/cps_data.csv'
cps_data = pd.read_csv(cps_data_url)
cps_data = cps_data.loc[cps_data['food_security_status'] != '-1']
cps_data = cps_data.loc[cps_data['food_security_status'] != '-9']

The data needs to be cleansed before data mutation

In [34]:
# create a function for age group
def age_group(x):
    if x < 13:
        return 'child'
    elif 13 <= x < 17:
        return 'teenager'
    elif 17 <= x <= 62:
        return 'adult'
    else:
        return 'senior'
    
cps_data_revised = cps_data.loc[cps_data['age'] != -1].drop(['Unnamed: 0'], axis=1)
cps_data_revised['age_group'] = cps_data_revised['age'].apply(age_group)

cps_data_revised

,household_id,household_id_2,age,sex,education,race,hispanic,family_income,household_size,housing_tenure,...,usual_weekly_food_spending,received_snap,received_school_lunch,received_school_breakfast,received_wic,person_supplement_weight,household_supplement_weight,state,poverty_indicator,age_group
0,70061014511774,16011,43,Female,Masters_degree,2,Non-Hispanic,"$75,000 to $99,999",5,Rented,...,280,-1,-1,-1,-1,3267.0314,3267.0314,Alabama,0,adult
1,70061014511774,16011,44,Male,High_school_graduate_diploma_or_GED,2,Non-Hispanic,"$75,000 to $99,999",5,Rented,...,280,-1,-1,-1,-1,3227.4523,3267.0314,Alabama,0,adult
2,70061014511774,16011,13,Female,Not_relevant,2,Non-Hispanic,"$75,000 to $99,999",5,Rented,...,280,-1,-1,-1,-1,4634.0388,3267.0314,Alabama,0,teenager
3,70061014511774,16011,15,Female,9th_grade,2,Non-Hispanic,"$75,000 to $99,999",5,Rented,...,280,-1,-1,-1,-1,5084.4886,3267.0314,Alabama,0,teenager
4,70061014511774,16011,20,Female,High_school_graduate_diploma_or_GED,2,Non-Hispanic,"$75,000 to $99,999",5,Rented,...,280,-1,-1,-1,-1,5443.0958,3267.0314,Alabama,0,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126806,655340013741005,15111,49,Female,Some_college_no_degree,1,Non-Hispanic,"$75,000 to $99,999",2,Owned/Being Bought,...,300,-1,-1,-1,-1,654.9122,654.9122,Wyoming,0,adult
126807,655340013741005,15111,54,Male,Some_college_no_degree,1,Non-Hispanic,"$75,000 to $99,999",2,Owned/Being Bought,...,300,-1,-1,-1,-1,637.8705,654.9122,Wyoming,0,adult
126809,655360677191105,15111,51,Male,High_school_graduate_diploma_or_GED,1,Non-Hispanic,"$30,000 to $34,999",1,Owned/Being Bought,...,110,No,-1,-1,-1,442.6095,442.6095,Wyoming,1,adult
126812,655907114899590,16111,52,Male,High_school_graduate_diploma_or_GED,1,Non-Hispanic,"$100,000 to $149,999",2,Owned/Being Bought,...,100,-1,-1,-1,-1,472.5786,383.4010,Wyoming,0,adult


In [43]:
# low food security data
low_food_security_data = cps_data_revised[cps_data_revised['food_security_status'] != 'Food Secure']
low_food_security_count = low_food_security_data.groupby(['state', 'age_group', 'sex'])['food_security_status'].count().reset_index(name='low_food_security_count')
low_food_security_count
# cps_data_revised.to_excel('Resources/cps_data_revised.xlsx')

,state,age_group,sex,low_food_security_count
0,Alabama,adult,Female,50
1,Alabama,adult,Male,36
2,Alabama,child,Female,16
3,Alabama,child,Male,8
4,Alabama,senior,Female,17
...,...,...,...,...
396,Wyoming,child,Male,26
397,Wyoming,senior,Female,17
398,Wyoming,senior,Male,14
399,Wyoming,teenager,Female,5
